In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# !mkdir dataset
# !pip install gdown
# !gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
# !mv refcocog.tar.gz ./dataset/
# !ls dataset

In [ ]:
# !gdown 1UmsNnuEs9kdstMs1t2NNmYhFZCC0C7Zq

In [ ]:
import tarfile
# data_dir='/content/dataset/refcocog.tar.gz'
data_dir='/content/drive/MyDrive/refcocog.tar.gz'
# Extract data
tar = tarfile.open(data_dir)
tar.extractall()

In [ ]:
pip install git+https://github.com/openai/CLIP.git

In [ ]:
!git clone https://github.com/ultralytics/yolov5

##Baseline

In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
!pip install transformers

All dependency or packages import

In [ ]:
import cv2
import os, clip
from PIL import Image
import torch
import torch.nn as nn
import pickle, json
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision
from torchvision.utils import draw_bounding_boxes
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import torchvision.models as models
import torch.nn.functional as F
import albumentations as A
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
transform = A.Compose([
                A.Resize(640, 640, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ])
class RefCOCOgDataset(Dataset):
  def __init__(self, transform = None):
    super(RefCOCOgDataset, self).__init__()
    self.images = []
    self.descriptions = []
    self.bboxes = []

    f = open('refcocog/annotations/refs(umd).p', 'rb')
    self.data = pickle.load(f)

    instances_file = os.path.join('refcocog/annotations/instances.json')
    instances = json.load(open(instances_file, 'r'))
    self.insta = instances
    cnt = 0
    for item in self.data:
      self.descriptions.append(item['sentences'])
      cnt = cnt+1
      for c in instances['images']:
        if c['id'] == item['image_id']:
            self.images.append(c['file_name'])
            break
      for c in instances['annotations']:
        if c['id'] == item['ann_id']:
            self.bboxes.append(c['bbox'])
            break
      if cnt == 100:
        break
    self.transform = transform

  def __getitem__(self, idx):

    img_path = f'refcocog/images/{self.images[idx]}'
    img_0 = cv2.imread(img_path)
    # shape_0 = torch.tensor(list(img_0.shape))
    # image = cv2.cvtColor(img_0, cv2.COLOR_BGR2RGB)
    # image = self.transform(image=img_0)['image']
    # clip_image = torch.tensor(image).permute(2, 0, 1).float()
    labels = [x['raw'] for x in self.descriptions[idx]]
    text_inputs = clip.tokenize(f"a photo of a {labels[0]}")
    bbox = torch.tensor(self.bboxes[idx])
    return img_0, text_inputs[0], bbox

  def __len__(self):

    return len(self.images)


In [ ]:
import torch

def calculate_iou(pred_boxes, target_boxes):

    x1 = torch.max(pred_boxes[:, 0], target_boxes[:, 0])
    y1 = torch.max(pred_boxes[:, 1], target_boxes[:, 1])
    x2 = torch.min(pred_boxes[:, 2], target_boxes[:, 2])
    y2 = torch.min(pred_boxes[:, 3], target_boxes[:, 3])

    intersection = torch.clamp((x2 - x1 + 1), min=0) * torch.clamp((y2 - y1 + 1), min=0)

    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0] + 1) * (pred_boxes[:, 3] - pred_boxes[:, 1] + 1)
    target_area = (target_boxes[:, 2] - target_boxes[:, 0] + 1) * (target_boxes[:, 3] - target_boxes[:, 1] + 1)
    union = pred_area + target_area - intersection

    iou = intersection / union

    return iou

In [ ]:
dataset = RefCOCOgDataset(transform = transform)

In [ ]:
from torchvision import transforms
to_pil = transforms.ToPILImage()
def baseline(x):
    image = x[0]
    captions = x[1]
    result = yolo(image)
    bboxes = result.crop(save=False)
    im_features = []
    for bbox in bboxes:
        image_input = preprocess(Image.fromarray(bbox['im'])).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        im_features.append(image_features)
    if len(bboxes) == 0:
      return 0, torch.zeros(4)
    text_inputs = captions.unsqueeze(0).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)

    text_features /= text_features.norm(dim=-1, keepdim=True)
    im_features = torch.stack(im_features)
    similarity = (100.0 * im_features @ text_features.T).softmax(dim=0)
    top_values, top_indices = similarity.topk(1, dim=0)
    return top_values.item(), bboxes[top_indices]['box']

In [ ]:
def calculate_iou1d(box1, box2):
    # Calculate coordinates of intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute area of intersection
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Compute areas of the two bounding boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Compute IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou


In [ ]:
import numpy as np

def calculate_precision_recall(pred_boxes, true_boxes, iou_threshold=0.5):
    sorted_indices = np.argsort(pred_boxes[:, -1])[::-1]
    pred_boxes = pred_boxes[sorted_indices]

    num_predictions = len(pred_boxes)
    num_targets = len(true_boxes)

    true_positives = np.zeros(num_predictions)
    false_positives = np.zeros(num_predictions)
    false_negatives = np.zeros(num_targets)

    for i, pred_box in enumerate(pred_boxes):
        best_iou = 0
        best_match_index = -1

        for j, true_box in enumerate(true_boxes):
            iou = calculate_iou1d(pred_box, true_box)

            if iou > best_iou and iou >= iou_threshold:
                best_iou = iou
                best_match_index = j

        if best_match_index >= 0:
            true_positives[i] = 1
            false_negatives[best_match_index] = 1
        else:
            false_positives[i] = 1

    cum_true_positives = np.cumsum(true_positives)
    cum_false_positives = np.cumsum(false_positives)
    cum_false_negatives = np.cumsum(false_negatives)

    precision = cum_true_positives / (cum_true_positives + cum_false_positives)
    recall = cum_true_positives / (cum_true_positives + cum_false_negatives)

    return precision, recall

def calculate_ap(precision, recall):
    recall = np.concatenate(([0], recall, [1]))
    precision = np.concatenate(([0], precision, [0]))

    for i in range(precision.size - 1, 0, -1):
        precision[i - 1] = np.maximum(precision[i - 1], precision[i])

    indices = np.where(recall[1:] != recall[:-1])[0] + 1
    ap = np.sum((recall[indices] - recall[indices - 1]) * precision[indices])

    return ap

def calculate_mAP(ap_values):
    mAP = np.mean(ap_values)
    return mAP

In [ ]:
pred_boxes = []
true_boxes = []

for idx, x in enumerate(dataset):
  print(f"\nTop predictions for {dataset.descriptions[idx][0]['raw']}:-------------------------\n")
  conf, box = baseline(x)
  print("target:", x[2])
  print("confidence: ", 100*conf)
  print("box: ", box)
  if conf != 0:
    x1,y1, x2, y2 = box
    pred_boxes.append([x1.item(), y1.item(), x2.item(), y2.item()])
    true_boxes.append(x[2].tolist())

In [ ]:
pred_boxes = np.array(pred_boxes)
true_boxes = np.array(true_boxes)

In [ ]:
precision, recall = calculate_precision_recall(pred_boxes, true_boxes)
ap = calculate_ap(precision, recall)
mAP = calculate_mAP(ap)

print("AP values:", ap)
print("mAP:", mAP)

# Finetuning starts here

### Custom dataset for finetuning

In [ ]:
transform = A.Compose([
                A.Resize(640, 640, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ])
class RefCOCOgDataset(Dataset):
  def __init__(self, transform = None):
    super(RefCOCOgDataset, self).__init__()
    self.images = []
    self.descriptions = []
    self.bboxes = []

    f = open('refcocog/annotations/refs(umd).p', 'rb')
    self.data = pickle.load(f)

    instances_file = os.path.join('refcocog/annotations/instances.json')
    instances = json.load(open(instances_file, 'r'))
    self.insta = instances
    cnt = 0
    for item in self.data:
      self.descriptions.append(item['sentences'])
      cnt = cnt+1
      for c in instances['images']:
        if c['id'] == item['image_id']:
            self.images.append(c['file_name'])
            break
      for c in instances['annotations']:
        if c['id'] == item['ann_id']:
            self.bboxes.append(c['bbox'])
            break
      if cnt == 100:
        break
    self.transform = transform

  def __getitem__(self, idx):

    img_path = f'refcocog/images/{self.images[idx]}'
    img_0 = cv2.imread(img_path)
    shape_0 = torch.tensor(list(img_0.shape))
    image = cv2.cvtColor(img_0, cv2.COLOR_BGR2RGB)
    image = self.transform(image=image)['image']
    clip_image = torch.tensor(image).permute(2, 0, 1).float()
    labels = [x['raw'] for x in self.descriptions[idx]]
    text_inputs = clip.tokenize(f"a photo of a {labels[0]}")
    bbox = torch.tensor(self.bboxes[idx])
    return clip_image, text_inputs[0], bbox

  def __len__(self):
    return len(self.images)

In [ ]:
def get_data(batch_size=15, test_batch_size=15):
  full_training_data = RefCOCOgDataset(transform=transform)
  num_samples = len(full_training_data)
  training_samples = int(num_samples * 0.5 + 1)
  validation_samples = num_samples - training_samples
  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=2)
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=2)
  return train_loader, val_loader

In [ ]:
da = get_data()

## Prepare model

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from transformers import ViTModel
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

class ViTBBModel(nn.Module):
    def __init__(self, num_boxes, hidden_dim=512):
        super(ViTBBModel, self).__init__()

        self.backbone = ViTModel.from_pretrained("google/vit-base-patch16-224")
        self.ln = nn.Linear(self.backbone.config.hidden_size, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.ln2 = nn.Linear(hidden_dim, 4 * num_boxes)

        self.mlp_regressor = nn.Sequential(
            nn.Linear(self.backbone.config.hidden_size, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, 4 * num_boxes)
        )

    def forward(self, x):
        features = self.backbone(x).last_hidden_state
        f = self.ln(features[:,0])
        f = self.relu(f)
        f = self.ln2(f)
        return f

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
clip_model, _ = clip.load("RN50")

In [ ]:
test_dataset = RefCOCOgDataset(transform=transform)

In [ ]:
import numpy as np

def non_max_suppression(yolo_output, confidence_threshold, iou_threshold):
    # Extract bounding box coordinates, class predictions, and confidence scores from the YOLO output
    boxes = yolo_output[..., :4]
    class_predictions = yolo_output[..., 5:]
    confidence_scores = yolo_output[..., 4]

    selected_boxes = []
    for box, class_probs, confidence_scores in zip(boxes, class_predictions, confidence_scores):
        # Apply confidence thresholding
        mask = confidence_scores >= confidence_threshold
        box = box[mask].cpu()
        class_probs = class_probs[mask].cpu()
        confidence_scores = confidence_scores[mask].cpu()

        # Apply non-maximum suppression
        selected_indices = []
        while len(confidence_scores) > 0:
            # Select the box with the highest confidence score
            best_index = np.argmax(confidence_scores)
            selected_indices.append(best_index)

            # Calculate IoU between the best box and the rest
            ious = calculate_iou(box[best_index], box[selected_indices[:-1]])

            # Remove indices of boxes with IoU greater than the threshold
            indices_to_remove = [i for i, iou in enumerate(ious) if iou > iou_threshold]
            box = np.delete(box, indices_to_remove, axis=0)
            class_probs = np.delete(class_probs, indices_to_remove, axis=0)
            confidence_scores = np.delete(confidence_scores.cpu(), indices_to_remove, axis=0)

        # Collect the selected boxes along with class IDs and confidence scores
        selected_boxes.extend([(b[0], b[1], b[2], b[3], np.argmax(c), s) for b, c, s in zip(box, class_probs, confidence_scores)])

    return selected_boxes

def calculate_iou(box1, boxes2):
    x1 = np.maximum(box1[0], boxes2[0])
    y1 = np.maximum(box1[1], boxes2[1])
    x2 = np.minimum(box1[2], boxes2[2])
    y2 = np.minimum(box1[3], boxes2[3])

    intersection = np.maximum(0, x2 - x1 + 1) * np.maximum(0, y2 - y1 + 1)
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_boxes2 = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)
    union = area_box1 + area_boxes2 - intersection

    iou = intersection / union
    return iou


In [ ]:
# from yolov5.utils.general import (non_max_suppression, scale_boxes)
import torch.nn.functional as F

class CustomClip(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.detector = model
        self.clip_model = clip_model
    def givemebox(self, img, bbox):
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      t = torch.stack((img[0][0][y1:y2, x1:x2], img[0][1][y1:y2, x1:x2], img[0][2][y1:y2, x1:x2]))
      t = to_pil(t)
      t = preprocess(t).unsqueeze(0).to(device)
      return t
    def forward(self, x):
        similar = []
        clip_image = x[0].to(device)
        captions = x[1].to(device)
        im_features = []
        results = self.detector(clip_image, size=640)
        results = non_max_suppression(results, 0.2, 0.2)
        for det in results:
          for *xyxy, conf, cls in det:
            try:
              t = self.givemebox(clip_image, xyxy)
            except Exception as e:
              continue
            with torch.no_grad():
                image_features = self.clip_model.encode_image(t)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            im_features.append(image_features)
        im_features = torch.stack(im_features)

        with torch.no_grad():
          text_features = self.clip_model.encode_text(captions)
          text_features /= text_features.norm(dim=-1, keepdim=True)
        print(im_features.shape)
        similarity = (100.0 * im_features @ text_features.T).softmax(dim=0)
        top_values, top_indices = similarity.topk(1, dim=0)
        print(top_indices)
        print(len(results))
        return results[top_indices]

x = test_dataset[0]

net = CustomClip()
res = net((x[0].unsqueeze(0), x[1].unsqueeze(0)))
res

In [ ]:
import torch
import torch.nn as nn

class IoULoss(nn.Module):
    def __init__(self):
        super(IoULoss, self).__init__()

    def forward(self, pred_boxes, target_boxes):
        batch_size, num_boxes = pred_boxes.size()

        iou = calculate_iou(pred_boxes, target_boxes)
        loss = 1 - iou
        return loss.mean()


In [ ]:
def get_cost_function():
  loss = IoULoss()
  return loss

In [ ]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.parameters(), 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)

  return optimizer

In [ ]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
  samples = 0.0
  cumulative_loss = 0.0
  # set the network to training mode
  cumulative_cIoU = 0.0
  cumulative_count = 0.0
  net.train()

  # iterate over the training set
  for batch_idx, (img, label, box) in enumerate(data_loader):
      inputs = img.to(device)
      box = box.to(device)
      label = label.to(device)
      outputs = net((inputs, label))
      loss = cost_function(outputs, box)
      samples += inputs.shape[0]
      cumulative_loss += loss
      box[batch_idx].to(device)
      cIoU = calculate_iou(outputs, box)
      cumulative_cIoU += cIoU.sum().item()
      cumulative_count += len(outputs)
      # print(cumulative_cIoU)
      # print(cumulative_count)
      torch.cuda.empty_cache()
  return cumulative_loss / samples, cumulative_cIoU / cumulative_count

def test_step(net, data_loader, cost_function, device='cuda'):
  samples = 0.0
  cumulative_loss = 0.0
  cumulative_cIoU = 0.0
  cumulative_count = 0.0

  net.eval()
  with torch.no_grad():
    for batch_idx, (img, label, box) in enumerate(data_loader):
      inputs = img.to(device)
      box = box.to(device)
      label = label.to(device)
      outputs = net((inputs, label))
      loss = cost_function(outputs, box)

      samples += inputs.shape[0]
      cumulative_loss += loss

      cIoU = calculate_iou(outputs, box)
      cumulative_cIoU += cIoU.sum().item()
      cumulative_count += len(outputs)
      # print(cumulative_cIoU)
      # print(cumulative_count)
      torch.cuda.empty_cache()
  return cumulative_loss / samples, cumulative_cIoU / cumulative_count

In [ ]:
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [ ]:
def main(batch_size=15,
         learning_rate=0.001,
         weight_decay=0.000001,
         momentum=0.9,
         epochs=20):
  writer = SummaryWriter(log_dir="runs/exp1")
  modified_model = CustomClip().to(device)
  # modified_model = ViTBBModel(num_boxes=1).to(device)
  # modified_model = AttentionClip().to(device)
  train_loader, val_loader = da

  # test_labels = torch.tensor([item[2] for item in test_dataset])
  # test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
  # test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)

  # define the cost function
  cost_function = get_cost_function()
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  # val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  # test_loss, test_accuracy = test_step(modified_model, val_loader, cost_function)

  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  # log_values(writer, -1, val_loss, val_accuracy, "validation")
  # log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  # print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  # print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):

    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    # val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)

    # logs to TensorBoard
    # log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    # print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  # compute final evaluation results
  print('After training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  # val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  # test_loss, test_accuracy = test_step(modified_model, val_loader, cost_function)

  # log to TensorBoard
  log_values(writer, epochs, train_loss, train_accuracy, "train")
  # log_values(writer, epochs, val_loss, val_accuracy, "validation")
  # log_values(writer, epochs, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  # print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  # print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
main()